In [ ]:
# Reinforcement Learning for Stock Trading with Free Data and Simulated Trading
# This script replaces Alpaca API with Yahoo Finance (yfinance) for historical data
# and simulates trading instead of using real API trades

# Install required libraries before running:
# pip install yfinance numpy pandas backtrader time requests_cache

import numpy as np
import pandas as pd
import yfinance as yf  # Free stock data source
import backtrader as bt  # Backtesting.py alternative
import time
import requests_cache

# Enable request caching to avoid rate limit issues
yf_session = requests_cache.CachedSession('yfinance_cache', expire_after=3600)

# Function to fetch historical stock data from Yahoo Finance
def get_stock_data(symbol, timeframe="1d", period="1y", max_retries=5, delay=10):
    """
    Fetches historical stock data from Yahoo Finance with retry mechanism and caching.

    Parameters:
    - symbol: Stock ticker (e.g., 'AAPL')
    - timeframe: Interval between data points (e.g., '1d', '1h')
    - period: How much data to fetch (e.g., '1y' for 1 year)
    - max_retries: Number of retries if rate limit error occurs
    - delay: Time to wait between retries (in seconds)

    Returns:
    - DataFrame with historical price data or None if request fails
    """
    for attempt in range(max_retries):
        try:
            df = yf.download(symbol, interval=timeframe, period=period, session=yf_session)
            if isinstance(df, tuple):  # Handle tuple issue
                print(f"Error fetching data for {symbol}: Response returned a tuple")
                return None
            if df.empty:
                print(f"Warning: No data found for {symbol}")
                return None
            return df
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    print(f"Failed to fetch data for {symbol} after {max_retries} attempts.")
    return None

# Simulated RL-based trading strategy (random for now, replace with trained model)
def rl_trading_strategy(state):
    """
    Reinforcement learning model placeholder.
    Returns: 0 (hold), 1 (buy), 2 (sell)
    """
    return np.random.choice([0, 1, 2])

# Simulated trade execution
def simulated_trade(symbol, qty, side="buy", portfolio={"cash": 10000, "holdings": {}}):
    """
    Simulates a stock trade without using a real API.
    """
    df = get_stock_data(symbol, timeframe="1d", period="1d")
    if df is None:
        print("Skipping trade due to missing data.")
        return portfolio
    
    price = df["Close"].iloc[-1]
    if side == "buy" and portfolio["cash"] >= price * qty:
        portfolio["cash"] -= price * qty
        portfolio["holdings"].setdefault(symbol, 0)
        portfolio["holdings"][symbol] += qty
        print(f"Simulated BUY: {qty} shares of {symbol} at ${price}")
    elif side == "sell" and portfolio["holdings"].get(symbol, 0) >= qty:
        portfolio["cash"] += price * qty
        portfolio["holdings"][symbol] -= qty
        print(f"Simulated SELL: {qty} shares of {symbol} at ${price}")
    else:
        print("Not enough cash or holdings for trade.")
    return portfolio

# Function to execute RL-based simulated trading
def trade_with_rl(symbol, shares=10, portfolio={"cash": 10000, "holdings": {}}):
    """
    Uses RL model to make simulated trades.
    """
    df = get_stock_data(symbol, timeframe="1d", period="10d")
    if df is None:
        print("Skipping trade due to missing data.")
        return
    
    state = np.array([df["Close"].values[-1]])  # Using last close price as state
    action = rl_trading_strategy(state)
    if action == 1:
        simulated_trade(symbol, qty=shares, side="buy", portfolio=portfolio)
    elif action == 2:
        simulated_trade(symbol, qty=shares, side="sell", portfolio=portfolio)
    else:
        print(f"Holding {symbol}")

# Backtesting RL strategy using Backtrader
class RLTradingStrategy(bt.Strategy):
    def __init__(self):
        self.signal = np.random.choice([0, 1, -1], size=len(self.data))  # Placeholder for RL model decisions
    
    def next(self):
        if self.signal[self.data.index.get_loc(self.data.index[self.bar])] == 1:
            self.buy()
        elif self.signal[self.data.index.get_loc(self.data.index[self.bar])] == -1:
            self.sell()

# Fetch historical stock data for backtesting
df = get_stock_data("AAPL", timeframe="1d", period="1y")
if df is not None:
    # Convert price data to Backtrader format
    data = bt.feeds.PandasData(dataname=df)

    # Create and run backtest
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RLTradingStrategy)
    cerebro.adddata(data)
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)

    print("Starting Portfolio Value:", cerebro.broker.getvalue())
    cerebro.run()
    print("Ending Portfolio Value:", cerebro.broker.getvalue())

    # Plot results
    cerebro.plot()
else:
    print("Skipping backtest due to missing data.")

# Run Simulated Trading (Comment out if only running backtesting)
# trade_with_rl("AAPL", shares=10)